<a href="https://colab.research.google.com/github/A1exandr12/JB_Intern-WikiEd/blob/main/JetBrains_internship.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Скачиваем необходимые модули

In [ ]:
!pip install wget
!pip2 install Levenshtein 

Скачиваем архив Dump'а

In [ ]:
!wget https://dumps.wikimedia.org/enwiki/20210301/enwiki-20210301-pages-meta-history4.xml-p554267p558391.bz2

Скачиваем инструмент WikiEd и выполняем make

In [ ]:
!git clone https://github.com/snukky/wikiedits.git
!make 


Разархивируем данные, прогоним их через скрипт для вытаскивания полной информации, которую затем запишем в файл и будем работать с ним

In [ ]:
!bzcat /content/enwiki-20210301-pages-meta-history4.xml-p554267p558391.bz2 | python2 /content/drive/MyDrive/JetBrains_Intern/wikiedits/bin/wiki_edits.py -m > EN_pages-meta-history.txt

Скачиваем модуль ERRANT, который позволяет определять изменения между предложениями

In [ ]:
!pip3 install -U pip setuptools wheel
!pip3 install errant
!python3 -m spacy download en

Классификация граммматических исправлений

In [39]:
dataset_file = open("Dataset.txt", "w")  
i = 0
annotator = errant.load('en')
flag = 0
dataset = []
with open("EN_pages-meta-history.txt", "r") as file:
    for line in file:
      if '###' in line and 'grammar' in line.lower():
        flag = 1
      else:
        if '###' in line or line == '\n':
          pass
        else:
          if flag == 1 or flag == 2:
            if flag==1:
              prevline = line;
              flag = 2
            else:
              dataset_file.write(prevline) 
              dataset_file.write('\n')
              dataset_file.write(line) 
              dataset_file.write('\n')
              dataset_file.write('\n')  
              #dataset.append(prevline + '\n' + line)       
              #print(prevline)
              #print(line)
              #print('\n')
              flag = 0
          else:
            if i%2==1:
              orig = annotator.parse(prevline)
              cor = annotator.parse(line)
              edits = annotator.annotate(orig, cor)
              for e in edits:
                l = [(x in e.type) for x in ['ADV', 'DET', 'MORPH', 'INFL', 'NUM', 'POSS', 'PART', 'PREP', 'PRON', 'SPELL', 'SVA', 'TENSE']]
                if True in l: 
                  dataset_file.write(prevline) 
                  dataset_file.write('\n')
                  dataset_file.write(line) 
                  dataset_file.write('\n')
                  dataset_file.write('\n')  
                  #dataset.append(prevline + '\n' + line)
                  #print(prevline)
                  #print(line)
                  #print('\n')
                  break
            else:
              prevline = line
            i+=1
dataset_file.close()
      

In [ ]:
#from google.colab import files
#files.download('Dataset.txt')